In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)

In [3]:
# 批次大小
Batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=Batch_size,
                          shuffle=True)

# 装载训练集
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=Batch_size,
                          shuffle=True)

In [4]:
for i, data in enumerate(train_loader):
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
inputs.size()

torch.Size([64, 1, 28, 28])

In [6]:
# 定义网络结构
# input_size输入特征的大小
# hidden_size，LSTM模块数量
# num_layer,LSTM的层数
# LSTM默认input（seq_len,batch,feature）
# batch_first=Ture,input和output(batch,seq_len,feature)
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(
            input_size=28,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )
        self.out = torch.nn.Linear(in_features=64, out_features=10)
        self.softmax = torch.nn.Softmax(dim=1)

    # (batch, seq_len, frature)
    def forward(self, x):
        x = x.view(-1, 28, 28)
        # output:[batch, seq_len, hidden_size]包含每个序列的输出结果
        # 虽然LSTM的batch——first为Ture，但是h_n, c_n的第0维度还是num_layers
        # h_n:[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        # c_n:[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        output, (h_n, c_n) = self.lstm(x)
        output_in_last_timestep = h_n[-1, :, :]
        x = self.out(output_in_last_timestep)
        x = self.softmax(x)
        return x

In [7]:
LR = 0.001
model = LSTM()
#model.load_state_dict(torch.load('./model/my_model.pth'))
CrossEntropy_loss = nn.CrossEntropyLoss()
# 定义优化器，设置L2化
optimizer = optim.Adam(model.parameters(), LR)

In [8]:
def train():
    # 训练状态
    model.train()
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果(64, 10)
        out = model(inputs)
        # 交叉熵代价函数out(batch,C),labels(batch)
        loss = CrossEntropy_loss(out, labels)
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()


# 用于模型测试
def test():
    # 模型的测试状态（此状态下，模型的dropout层不起作用，所有神经元都要工作）
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果(64, 10)
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))

    # 同上，将训练集合测试集的准确率都表示出来
    correct = 0
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果(64, 10)
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item()/len(train_dataset)))

#### 小结：
> 手动打出这个训练流程你会发现相对于keras这种高度集成的API，torch需要从头到尾的用代码去实现神经网络的训练过程，所以其中有很多细节可以修改，当然之后可以尝试学习autoML，可以更快的调参

> 可以看出pytorch和tensorflow各有各的好处，比如tensorboard的使用，先别管先把手上的一些小任务小领域先完成

In [9]:
for epoch in range(10):
    print('epoch:', epoch)
    train()
    test()

epoch: 0
Test acc:0.8298
Train acc:0.8314666666666667
epoch: 1
Test acc:0.939
Train acc:0.9388333333333333
epoch: 2
Test acc:0.9527
Train acc:0.9546
epoch: 3
Test acc:0.8961
Train acc:0.9007166666666667
epoch: 4
Test acc:0.9653
Train acc:0.9662833333333334
epoch: 5
Test acc:0.9617
Train acc:0.9667333333333333
epoch: 6
Test acc:0.9703
Train acc:0.9724333333333334
epoch: 7
Test acc:0.9633
Train acc:0.9659666666666666
epoch: 8
Test acc:0.9624
Train acc:0.9625666666666667
epoch: 9
Test acc:0.9721
Train acc:0.97405


In [12]:
# 模型的保存
torch.save(model.state_dict(), './model/my_model.pth')

Test acc:0.9721
Train acc:0.97405
